In [ ]:
import librosa
import numpy as np

import shutil
import os
import zipfile
import tarfile

import time
import random

import wave

from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
path_to_zip = '/content/drive/My Drive/datasets/Audio_Speech_Actors_01-24.zip'
target_path = '/content/RAVDESS'
path_to_extract_to = os.path.join(target_path, 'extracted')

if not os.path.isdir(target_path):
    os.mkdir(target_path)

path_to_zip = shutil.copy2(path_to_zip, target_path)

print('Zip archive is copied')

with zipfile.ZipFile(path_to_zip, 'r') as zip_ref:
    zip_ref.extractall(path_to_extract_to)

print('Zip archive is unpacked')

Zip archive is copied
Zip archive is unpached


In [ ]:
def get_paths_to_wavs(path_to_dataset):
    file_paths_list = []

    for root, dirs, files in os.walk(path_to_dataset):
        if len(files) != 0:
            file_paths_list += [os.path.join(root, f) for f in files if f.endswith('.wav')]

    return file_paths_list

In [ ]:
# prepare numpy RAVDESS dataset
paths_to_wavs_list = get_paths_to_wavs(path_to_extract_to)

path_to_save = 'npy'

if not os.path.isdir(path_to_save):
    os.mkdir(path_to_save)

for path_to_wav in paths_to_wavs_list:
    # read file
    wav, sr = librosa.load(path_to_wav)
    wav.astype(np.float32)

    # Parse the filename, which has the following pattern:
    # modality-vocal_channel-emotion-intensity-statement-repetition-actor.wav
    file_name = os.path.split(path_to_wav)[1]
    file_name = file_name[:-4]

    np.save(os.path.join(path_to_save, file_name + '.npy'), np.array([wav, sr]))

print('Numpy converting process is completed')

# archive npy-s files
archive_name = 'ravdess-speech-sr{}.tar.xz'.format(sr)

with tarfile.open(archive_name, 'w:xz') as tar:
    for name in os.listdir(path_to_save):
        tar.add(os.path.join(path_to_save, name))
print('tar.xz archive packing process is completed')

path_to_zip = shutil.copy2(archive_name, '/content/drive/My Drive/datasets/')

print('The archive is stored to disk')

Numpy converting process is completed
tar.xz archive packing process is completed
The archive is stored to disk
